In [ ]:
from neural_poisson.model.neural_poisson import NeuralPoisson

ckpt_path = "/home/borth/2d-gaussian-splatting/logs/2025-02-19/14-49-43/checkpoints/epoch_339.ckpt"
model = NeuralPoisson.load_from_checkpoint(ckpt_path)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from pytorch3d.ops.marching_cubes import marching_cubes
import open3d as o3d

# Generate a 3D volumetric grid (e.g., a sphere)
grid_size = 32
x = torch.linspace(-1, 1, grid_size)
y = torch.linspace(-1, 1, grid_size)
z = torch.linspace(-1, 1, grid_size)
X, Y, Z = torch.meshgrid(x, y, z, indexing='ij')
vol = X**2 + Y**2 + Z**2 - 0.5**2  # Implicit surface of a sphere

# Apply marching cubes
verts, faces = marching_cubes(vol[None], isolevel=0.5)

# Convert to NumPy for visualization
verts = verts[0].numpy()
faces = faces[0].numpy()

# Create an Open3D mesh
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts)
mesh.triangles = o3d.utility.Vector3iVector(faces)
mesh.compute_vertex_normals()
o3d.visualization.draw_plotly([mesh])


In [ ]:
import wandb
wandb.init(project="thesis", entity="robinborth")

In [ ]:

v = (vol[None].permute(1, 0, 2, 3).expand(32, 3, 32, 32) > 0).to(torch.uint8) * 255
video = wandb.Video(v, fps=20, format="gif")
wandb.log({"video3": video})

In [ ]:
v.max()

In [ ]:
vol.shape

In [ ]:
import torch
ckpt = torch.load("/home/borth/2d-gaussian-splatting/test.ckpt")
ckpt

In [ ]:
import torch
from neural_poisson.model.encoder import MLP

mlp = MLP()
points = torch.rand((1000, 3))
torch.nn.init.uniform_(points, -1.0, 1.0)
mlp(points).mean()

In [ ]:
import torch

axis = "x"

grid_vals = torch.linspace(-1.0, 1.0, 256)
xs, ys = torch.meshgrid(grid_vals, grid_vals, indexing="ij")
zs = torch.zeros_like(xs)
if axis == "x":
    coords = (zs.ravel(), xs.ravel(), ys.ravel())
if axis == "y":
    coords = (xs.ravel(), zs.ravel(), ys.ravel())
if axis == "z":
    coords = (xs.ravel(), ys.ravel(), zs.ravel())
grid = torch.stack(coords).reshape(-1, 3)
x, _ = self.forward(points.to(self.device))

xs